In [1]:
pwd

'/content'

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras as keras

In [4]:
imdb_reviews = pd.read_csv("/content/imdb_reviews.csv")
test_reviews = pd.read_csv("/content/test_reviews.csv")

In [9]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


## map each word to distinct integers (hasing)

In [10]:
word_index = pd.read_csv("/content/word_indexes.csv")

In [11]:
word_index.head(10)

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411
5,spiders,16118
6,hanging,2348
7,woody,2292
8,trawling,52011
9,hold's,52012


In [13]:
word_index = dict(zip(word_index.Words,word_index.Indexes))

### map extra words in reviews

In [22]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [23]:
def review_encoder(text):
  arr = [word_index[word] for word in text]
  return arr

In [24]:
train_data, train_labels = imdb_reviews['Reviews'], imdb_reviews['Sentiment']
test_data, test_labels = test_reviews['Reviews'], test_reviews['Sentiment']

### Split on basis of white spaces(words)

In [25]:
train_data = train_data.apply(lambda review:review.split())
test_data = test_data.apply(lambda review:review.split())

In [26]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [28]:
def encode_sentiments(sentiments):
  if sentiments=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [29]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [30]:
  model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                          keras.layers.GlobalAveragePooling1D(),
                          keras.layers.Dense(16,activation='relu'),
                          keras.layers.Dense(1,activation='sigmoid')
                          ])

In [31]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [32]:
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 5s 71ms/step - loss: 0.6920 - accuracy: 0.5747 - val_loss: 0.6903 - val_accuracy: 0.6763
Epoch 2/30
49/49 [==============================] - 3s 64ms/step - loss: 0.6856 - accuracy: 0.7039 - val_loss: 0.6796 - val_accuracy: 0.6922
Epoch 3/30
49/49 [==============================] - 3s 65ms/step - loss: 0.6666 - accuracy: 0.7197 - val_loss: 0.6538 - val_accuracy: 0.7535
Epoch 4/30
49/49 [==============================] - 3s 64ms/step - loss: 0.6294 - accuracy: 0.7688 - val_loss: 0.6114 - val_accuracy: 0.7770
Epoch 5/30
49/49 [==============================] - 3s 64ms/step - loss: 0.5760 - accuracy: 0.8016 - val_loss: 0.5587 - val_accuracy: 0.8020
Epoch 6/30
49/49 [==============================] - 3s 62ms/step - loss: 0.5167 - accuracy: 0.8258 - val_loss: 0.5059 - val_accuracy: 0.8209
Epoch 7/30
49/49 [==============================] - 3s 62ms/step - loss: 0.4600 - accuracy: 0.8470 - val_loss: 0.4589 - val_accuracy: 0.8366
Epoch 8/30
49

In [33]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 3s 4ms/step - loss: 0.2894 - accuracy: 0.8857


In [44]:
index=np.random.randint(1,1000)


In [45]:
user_review=test_reviews.loc[index]
print(user_review)

Reviews      <START definitely not a film to <UNK> the indu...
Sentiment                                             positive
Name: 257, dtype: object


In [46]:
user_review=test_data[index]
user_review=np.array([user_review])
if(model.predict(user_review)>0.5).astype("int32"):
  print("Positive Sentiment")
else:
  print("Negative Sentiment")

Positive Sentiment
